In [1]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import tensorflow as tf
from torchvision import transforms
import torchvision

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset,TensorDataset
import torch.nn.functional as F
import torch.optim as optim
import pdb
from sklearn.model_selection import train_test_split


In [2]:
cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
TEST_DIR = '/content/drive/My Drive/Brain Tumor Classification Project/dataSet/Testing' # test data folder
TRAIN_DIR = '/content/drive/My Drive/Brain Tumor Classification Project/dataSet/Training' # train data folder
CATEGORIES = ["glioma","meningioma","notumor","pituitary"]

# Hyperparameters
hyparams = {
      "img_shape": (224,224),
      "n_class": 4,
      "learning_rate": 0.001,
      "batch_size": 16,
      "epochs": 10
  }


In [5]:

def one_hot_encoder(class_num):
    one_hot = torch.zeros(4)
    one_hot[class_num] = 1
    return one_hot


In [6]:

transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Resize(hyparams["img_shape"], antialias=True),  # Explicitly set antialias to True
      # Convert image to tensor
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize if needed
])
def load_data(dir,device):
    data = []
    for category in CATEGORIES:
        path = os.path.join(dir,category)  # create path
        class_num = torch.tensor(CATEGORIES.index(category))  # get the classification
        class_one_hot_vec = one_hot_encoder(class_num)
        for img in tqdm(os.listdir(path)):

          image = cv2.imread(os.path.join(path,img))  # convert to array and read grayscale images - 1 chanel
          new_array = transform(np.array(image)) # resize to normalize data size
          data.append([new_array, class_one_hot_vec])  # add this to our training_data

    return data



In [ ]:

data = load_data(TRAIN_DIR,device)
train_data, val_data = train_test_split(data, test_size=0.2)

test_data = load_data(TEST_DIR,device)

train_data = DataLoader(dataset=train_data, batch_size=hyparams["batch_size"], shuffle=True)
val_data = DataLoader(dataset=val_data, batch_size=hyparams["batch_size"], shuffle=True)
test_data = DataLoader(dataset=test_data, batch_size=hyparams["batch_size"], shuffle=True)


 34%|███▍      | 453/1333 [01:12<01:16, 11.55it/s]

In [ ]:
# prompt: plot 4*4 images for train_data


# Get a batch of training data
images, labels = next(iter(train_data))

# Plot the images in a 4x4 grid
plt.figure(figsize=(10, 10))
for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.imshow(images[i].permute(1, 2, 0))
    plt.title(CATEGORIES[labels[i].argmax()])
    plt.axis('off')

plt.show()


In [ ]:

class ResNet18(nn.Module):
    def __init__(self, num_classes=4):
        super(ResNet18, self).__init__()

        # Load the pre-trained ResNet18 model
        self.base_model = torchvision.models.resnet18(pretrained=True)

        # Modify the last fully connected layer to match the number of classes
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, num_classes)

    def forward(self, x):
        # Pass the input through the base model
        x = self.base_model(x)

        # Apply softmax activation function
        x = F.softmax(x, dim=1)

        # Return the output
        return x



In [ ]:

model = ResNet18().to(device)
optimizer = optim.Adam(model.parameters(), lr=hyparams["learning_rate"])
criterion = nn.CrossEntropyLoss()


In [ ]:


def train(train_data, model, criterion, optimizer):
    train_error = 0.0
    correct = 0
    total = 0

    model.train()

    for i, data in enumerate(tqdm(train_data)): # tqdm for Progress bar
        images, labels = data

        images = images.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        labels = labels.to(device)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_error += loss.item() * images.size(0) # gives the total contribution of the loss from all images in the batch

        _, predicted = torch.max(outputs.data, 1)

        for i,pred in enumerate(predicted):
            if labels[i][pred] == 1:
              correct += 1


    return train_error / len(train_data.dataset), correct / len(train_data.dataset)



In [ ]:
def test(test_data, model, criterion):
    test_error = 0.0
    correct = 0
    total = 0

    model.eval()

    with torch.no_grad():
        for i, data in enumerate(tqdm(test_data)): # tqdm for Progress bar
            images, labels = data

            images = images.to(device)
            outputs = model(images)
            labels = labels.to(device)
            loss = criterion(outputs, labels)
            test_error += loss.item() * images.size(0) # gives the total contribution of the loss from all images in the batch

            _, predicted = torch.max(outputs.data, 1)

            for i,pred in enumerate(predicted):
              if labels[i][pred] == 1:
                correct += 1


    return test_error / len(test_data.dataset), correct / len(test_data.dataset)



In [ ]:
for epoch in range(1, hyparams['epochs'] + 1): # start for 1 - for nicer log
    # train
    train_error,train_accuracy = train(train_data, model, criterion, optimizer)
    print("train_error " + str(train_error))
    print("train accuracy " + str(train_accuracy))
    # validate
    val_error,val_accuracy = test(val_data, model, criterion)
    print("val_error " + str(val_error))
    print("val accuracy " + str(val_accuracy))